## GPT-2

In [10]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained GPT-2 model and tokenizer

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

model = GPT2LMHeadModel.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\likhi\Documents\01 Pycharm Code Folder\edhec\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\likhi\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [11]:
def generate_response(prompt, max_length=50):
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # Generate response
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=50256)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [12]:
print("Chatbot: Hi there! How can I help you?")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break

    response = generate_response(user_input)
    print("Chatbot:", response)

Chatbot: Hi there! How can I help you?
Chatbot: Hi, how are you?

I'm a little bit of a nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a
Chatbot: good, but I'm not sure if it's a good idea to have a lot of people who are not going to be able to afford it.

"I think it's a good idea to have a lot of people who are not going
Chatbot: what the hell is going on here?"

"I'm not sure. I'm not sure what's going on here. I'm not sure what's going on here. I'm not sure what's going on here. I'm not sure
Chatbot: what are you talking about?"

"I'm talking about the fact that I'm a woman. I'm a woman. I'm a woman. I'm a woman. I'm a woman. I'm a woman. I'm a woman
Chatbot: cancel_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event_event
Chatbot: Goodbye!


In [13]:
"""
Adapted from:
https://www.machinecurve.com/index.php/2021/03/16/easy-chatbot-with-dialogpt-machine-learning-and-huggingface-transformers/
"""

from transformers import AutoModelForCausalLM, AutoTokenizer
import os

class DialoGPT:
    def __init__(
        self,
        model_name: str ='microsoft/DialoGPT-large',
    ):
        if not os.path.exists('./models/dialogpt'):
            AutoModelForCausalLM.from_pretrained(model_name).save_pretrained('./models/dialogpt')
            AutoTokenizer.from_pretrained(model_name).save_pretrained('./models/dialogpt')

        self.model = AutoModelForCausalLM.from_pretrained('./models/dialogpt')
        self.tokenizer = AutoTokenizer.from_pretrained('./models/dialogpt')

    def __call__(self, inputs: str) -> str:
        inputs_tokenized = self.tokenizer.encode(inputs+ self.tokenizer.eos_token, return_tensors='pt')
        reply_ids = self.model.generate(inputs_tokenized, max_length=1250, pad_token_id=self.tokenizer.eos_token_id)
        reply = self.tokenizer.decode(reply_ids[:, inputs_tokenized.shape[-1]:][0], skip_special_tokens=True)

        return reply

    def run(self):
        while True:
            user_input = input("User: ")
            print("Bot:", self(user_input))

In [ ]:
# bot = DialoGPT()
# bot.run()

## Dialog-GPT

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

In [12]:
model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-large')
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-large', use_fast=True,
                                          padding_side="left", add_eos_token=True, add_bos_token=True)

In [13]:
def generate_response(inputs):

    inputs_tokenized = tokenizer.encode(inputs + tokenizer.eos_token, return_tensors='pt')
    reply_ids = model.generate(inputs_tokenized, max_length=1250, pad_token_id=tokenizer.eos_token_id)
    reply = tokenizer.decode(reply_ids[:, inputs_tokenized.shape[-1]:][0], skip_special_tokens=True)
    
    return reply

In [ ]:
import time

print("Chatbot: Hi there! How can I help you?")
while True:
    user_input = input("You: ")
    print("You: ", user_input)
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break

    response = generate_response(user_input)
    print("Chatbot:", response)
    
    time.sleep(15)

Chatbot: Hi there! How can I help you?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


You:  hello,cna you help me?
Chatbot: I can't help you, I'm not a doctor.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


You:  you are rude bro
Chatbot: I am not your bro, pal.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


You:  ok
Chatbot: 


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


You:  



KeyboardInterrupt

